In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize

np.random.seed(42)

ImportError: cannot import name 'Estimator' from 'qiskit.primitives' (/Users/hexa_morph/Documents/SOC_QML/.venv/lib/python3.13/site-packages/qiskit/primitives/__init__.py)

In [ ]:
def create_qcnn_circuit(num_qubits):
    if not (num_qubits > 0 and (num_qubits & (num_qubits - 1) == 0)):
        raise ValueError("Number of qubits must be a power of 2.")

    # Parameters for encoding the data
    feature_params = ParameterVector('x', num_qubits)
    
    # Each layer has 2 params for convolution and 1 for pooling
    num_layers = int(np.log2(num_qubits))
    total_params = num_layers * 3
    circuit_params = ParameterVector('θ', total_params)
    
    qc = QuantumCircuit(num_qubits)
    param_idx = 0
    
    # Encode input data using RY rotations
    for i in range(num_qubits):
        qc.ry(feature_params[i], i)
    qc.barrier()

    active_qubits = list(range(num_qubits))
    while len(active_qubits) > 1:
        #Convolutional Layer
        for i in range(0, len(active_qubits) - 1, 2):
            qc.crz(circuit_params[param_idx], active_qubits[i], active_qubits[i+1])
            qc.crx(circuit_params[param_idx + 1], active_qubits[i], active_qubits[i+1])
        qc.barrier()

        #Pooling Layer
        new_active_qubits = []
        for i in range(0, len(active_qubits), 2):
            if i + 1 < len(active_qubits):
                # Apply a controlled gate to entangle info from the second qubit into the first
                qc.crx(circuit_params[param_idx + 2], active_qubits[i], active_qubits[i+1])
                new_active_qubits.append(active_qubits[i]) 
            else:
                new_active_qubits.append(active_qubits[i])
        
        active_qubits = new_active_qubits
        param_idx += 3
        qc.barrier()
        
    return qc, feature_params, circuit_params

In [ ]:
def generate_dataset(num_qubits, num_samples):
    X = []
    y = []
    
    for _ in range(num_samples // 2):
        # Class 0 (vertical) with noise
        base_v = np.array([np.pi/2, -np.pi/2] * (num_qubits // 2))
        X.append(base_v + np.random.normal(0, 0.1, num_qubits))
        y.append(0)

        # Class 1 (horizontal) with noise
        base_h = np.array([np.pi/2] * (num_qubits // 2) + [-np.pi/2] * (num_qubits // 2))
        X.append(base_h + np.random.normal(0, 0.1, num_qubits))
        y.append(1)
        
    return np.array(X), np.array(y)

In [ ]:





def main():
    NUM_QUBITS = 4
    NUM_SAMPLES = 40
    
    print("1. Creating QCNN circuit and dataset...")
    qcnn_circuit, feature_params, circuit_params = create_qcnn_circuit(NUM_QUBITS)
    X_train, y_train = generate_dataset(NUM_QUBITS, NUM_SAMPLES)
    
    y_train_mapped = 2 * y_train - 1

    estimator = Estimator()
    
    observable = SparsePauliOp("Z" + "I" * (NUM_QUBITS - 1))

    cost_history = []
    def cost_function(params):
        num_samples = len(X_train)
        
        tiled_circuit_params = np.tile(params, (num_samples, 1))
        
        param_values = np.hstack([X_train, tiled_circuit_params])
        
        job = estimator.run(
            circuits=[qcnn_circuit] * num_samples,
            observables=[observable] * num_samples,
            parameter_values=param_values
        )

        y_pred = job.result().values

        loss = np.mean((y_train_mapped - y_pred)**2)
        
        cost_history.append(loss)
        print(f"Loss: {loss:.6f}")
        return loss

    print("\n2. Starting QCNN training...")
    initial_params = np.random.uniform(0, 2 * np.pi, len(circuit_params))
    
    optimizer_result = minimize(cost_function, initial_params, method='COBYLA', options={'maxiter': 80})
    
    print("\n3. Training complete.")
    optimal_params = optimizer_result.x
    print(f"   - Final Loss: {optimizer_result.fun:.6f}")
    print(f"   - Optimal Parameters: {np.round(optimal_params, 4)}")

    print("\n4. Evaluating the trained model...")
    num_test_samples = len(X_train)
    tiled_optimal_params = np.tile(optimal_params, (num_test_samples, 1))
    test_param_values = np.hstack([X_train, tiled_optimal_params])
    
    job = estimator.run(
        circuits=[qcnn_circuit] * num_test_samples,
        observables=[observable] * num_test_samples,
        parameter_values=test_param_values
    )
    final_exp_vals = job.result().values
    
    predicted_labels = (final_exp_vals > 0).astype(int)
    
    accuracy = np.mean(predicted_labels == y_train)
    print(f"\n   - Final Accuracy on Training Data: {accuracy * 100:.2f}%")

    plt.figure(figsize=(10, 6))
    plt.plot(cost_history, color='royalblue', marker='o', linestyle='--')
    plt.title("QCNN Training Loss vs. Optimization Step")
    plt.xlabel("Optimization Step")
    plt.ylabel("Mean Squared Error (MSE) Loss")
    plt.grid(True)
    plt.show()


if __name__ == '__main__':
    main()